# Linear Regression
## Linear Regression

**Single and Multiple Variable Regression**

**Definition:** Single variable linear regression models the linear relationship between a dependent variable and a single independent variable.

$$
y = \beta_0 + \beta_1 x + \epsilon
$$

$$
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{bmatrix}
=
\begin{bmatrix}
1 & x_1 \\
1 & x_2 \\
\vdots & \vdots \\
1 & x_n
\end{bmatrix}
\begin{bmatrix}
\beta_0 \\
\beta_1
\end{bmatrix}
+
\begin{bmatrix}
\epsilon_1 \\
\epsilon_2 \\
\vdots \\
\epsilon_n
\end{bmatrix}
$$

**Definition:** Multiple variable regression models the relationship between a dependent variable and two or more independent variables.

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \ldots + \beta_k x_k + \epsilon
$$

$$
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{bmatrix}
=
\begin{bmatrix}
1 & x_{11} & x_{12} & \ldots & x_{1k} \\
1 & x_{21} & x_{22} & \ldots & x_{2k} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & x_{n1} & x_{n2} & \ldots & x_{nk}
\end{bmatrix}
\begin{bmatrix}
\beta_0 \\
\beta_1 \\
\vdots \\
\beta_k
\end{bmatrix}
+
\begin{bmatrix}
\epsilon_1 \\
\epsilon_2 \\
\vdots \\
\epsilon_n
\end{bmatrix}
$$


## OLS Estimator

**Definition:** The Ordinary Least Squares (OLS) estimator is used in linear regression to estimate the coefficients by minimizing the sum of squared residuals.

$$
\hat{\beta} = (X^\top X)^{-1} X^\top Y
$$

$$
SE(\hat{\beta}) = \sqrt{s^2 (X^\top X)^{-1}}
$$

**Standard Errors:**

1. Compute the residuals: $\epsilon = Y - X\hat{\beta}$.
2. Calculate the residual variance: $s^2 = \frac{\epsilon^\top \epsilon}{n - k}$, where $n$ is the number of observations and $k$ the number of parameters (including the intercept).
3. The variance-covariance matrix of $\hat{\beta}$ is $Var(\hat{\beta}) = s^2(X^\top X)^{-1}$.
4. The standard errors are the square roots of the diagonal elements of $Var(\hat{\beta})$.

Violations of OLS assumptions (like heteroscedasticity or autocorrelation) can lead to inaccurate standard errors, affecting inference. Robust standard errors can be used to correct for such violations.

**Robust Standard Errors:**

- Robust standard errors are adjusted to account for heteroskedasticity and potential autocorrelation, by weighting residuals differently and can also be extended to correct for autocorrelation.
- Calculation involves adjusting the variance-covariance matrix:  $Var_{robust}(\hat{\beta}) = (X^\top X)^{-1} X^\top S X (X^\top X)^{-1}$  where $S$ is a diagonal matrix with each element equal to the squared residual $\epsilon_i^2$.

**Properties:** OLS estimators are BLUE (Best Linear Unbiased Estimators) under the Gauss-Markov assumptions.

**Assumptions:**

1. Linearity: The relationship between dependent and independent variables is linear.
2. No Autocorrelation: The error terms are uncorrelated with each other (no serial correlation).
3. Homoskedasticity: Constant variance of error terms (no heteroskedasticity).
4. No Perfect Multicollinearity: Independent variables are not perfectly linearly related.
5. No Endogeneity: Error terms are uncorrelated with the independent variables. Weaker assumption **Zero conditional mean**, i.e., $E(\epsilon|X) = 0$

**Violation of OLS Assumption**  **Diagnostic Test**

- Non-linearity: Plotting residuals vs. fitted values, Ramsey's RESET test
- Endogeneity: Durbin-Wu-Hausman test
- Autocorrelation (Serial Correlation): Durbin-Watson test
- Heteroskedasticity: Breusch-Pagan test, White test
- Multicollinearity: Variance Inflation Factor (VIF)


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load Boston housing dataset
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=['MEDV'])

# Adding a constant to the model (for intercept)
X_const = sm.add_constant(X)

# OLS estimation
model = sm.OLS(y, X_const).fit()
print("OLS Estimation Results:")
print(model.summary())

# OLS with Robust Standard Errors
robust_model = model.get_robustcov_results()
print("\nOLS Estimation with Robust Standard Errors:")
print(robust_model.summary())

# Diagnostic tests
# 1. Multicollinearity: VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print("\nVariance Inflation Factor (VIF) for Multicollinearity:")
print(vif_data)

# 2. Heteroskedasticity: Breusch-Pagan and White tests
bp_test = het_breuschpagan(model.resid, model.model.exog)
white_test = het_white(model.resid, model.model.exog)
print("\nBreusch-Pagan test p-value:", bp_test[1])
print("White test p-value:", white_test[1])

# 3. Autocorrelation: Durbin-Watson test
dw_test = durbin_watson(model.resid)
print("\nDurbin-Watson test statistic:", dw_test)

# 4. Linearity: Ramsey's RESET test
# (Requires statsmodels 0.12.0 or later)
from statsmodels.stats.diagnostic import linear_reset
reset_test = linear_reset(model)
print("\nRamsey's RESET test p-value:", reset_test.pvalue)

# Optional: Plotting residuals vs. fitted values for visual inspection
plt.scatter(model.fittedvalues, model.resid)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Fitted Values')
plt.show()

## Binary and Multinomial Logit Model
The probability of choosing an option in a binary setting is modeled as:

$$
P(Y=1|X) = \frac{1}{1 + e^{-(\beta_0 + \beta_1X)}}
$$
- For multinomial choices:

$$
P(Y=j|X) = \frac{e^{\beta_j'X}}{\sum_{k=1}^Ke^{\beta_k'X}}
$$

- Logit Function:
The logit function is the core of these models. It maps a linear combination of explanatory variables to a probability. The function is given by:

$$
\text{Logit}(P) = \log\left(\frac{P}{1-P}\right)
$$

where $P$ is the probability of the occurrence of an event. It is the inverse of the logistic function and transforms probabilities into log-odds.

- Coefficient Interpretation:
In the binary logit model, the coefficients represent the log-odds of choosing one category over another. In the multinomial logit model, they represent the log-odds of choosing each alternative over a base category.

- Coefficient Estimation:
Coefficients are estimated using Maximum Likelihood Estimation (MLE). The likelihood function for the multinomial logit model is:

$$
L(\beta|X,Y) = \prod_{i=1}^n \left(\frac{e^{\beta_{Y_i}'X_i}}{\sum_{k=1}^Ke^{\beta_k'X_i}}\right)
$$

MLE finds the $\beta$ that maximizes this likelihood function.

- Fisher Information Matrix
The Fisher Information Matrix (FIM) is used to estimate the variance of MLE coefficients. It is defined as the negative expectation of the second derivative (Hessian) of the log-likelihood function:

$$
\mathcal{I}(\beta) = -E\left[\frac{\partial^2}{\partial \beta^2} \log L(\beta|X,Y)\right]
$$

The inverse of the FIM gives an estimate of the variance-covariance matrix of the estimated coefficients.

- Confidence Intervals
Confidence intervals for coefficients are constructed using the standard errors from the FIM. For a 95\% confidence interval:

$$
CI = \hat{\beta} \pm 1.96 \times SE(\hat{\beta})
$$

where $\hat{\beta}$ is the estimated coefficient and $SE(\hat{\beta})$ is its standard error.

- Use Case
Used in scenarios with two exclusive choices, like buy vs not buy, or applicable when individuals choose from more than two alternatives.



In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import statsmodels.api as sm  # Alternative using statsmodels

# Load Iris dataset
iris = datasets.load_iris()
X = iris.data
y = (iris.target == 1).astype(int)  # Binary target: 1 if 'versicolor', 0 otherwise

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and fit the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Performance Metrics
# Training performance
print("\nTraining Performance:")
print("Accuracy:", accuracy_score(y_train, y_pred_train))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))
print("Classification Report:\n", classification_report(y_train, y_pred_train))

# Testing performance
print("\nTesting Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Statsmodels implementation (for reference)
# X_train_const = sm.add_constant(X_train)
# X_test_const = sm.add_constant(X_test)
# sm_model = sm.Logit(y_train, X_train_const).fit()
# print(sm_model.summary())


##Binary and Multinomial Probit Model
Mathematical Representation:}
In the binary probit model, the probability of choosing an option is modeled as:
$$
P(Y=1|X) = \Phi(\beta_0 + \beta_1X)
$$
For multinomial probit, the model extends to multiple categories, with a similar structure.

- Probit Function
The probit function involves the cumulative distribution function (CDF) of the standard normal distribution, denoted by $\Phi$. It maps a linear combination of predictors to a probability:
$$
\text{Probit}(P) = \Phi^{-1}(P)
$$
where $\Phi^{-1}$ is the inverse CDF (quantile function) of the standard normal distribution.

- Coefficient Interpretation:
The coefficients in the probit model indicate the change in the z-score (from the standard normal distribution) for a one-unit change in the predictor variable.

- Coefficient Estimation:
Like the logit model, the probit model coefficients are estimated using Maximum Likelihood Estimation (MLE).

- Confidence Intervals:
Confidence intervals for probit model coefficients can be constructed using the estimated standard errors, similar to the logit model:
$$
CI = \hat{\beta} \pm z \times SE(\hat{\beta})
$$
where $z$ is the z-value from the standard normal distribution corresponding to the desired confidence level.

- Use Case:
The probit model is particularly useful in scenarios where the underlying process of choice is assumed to follow a normal distribution, such as in certain psychological measurements, voting behavior, etc.


In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import statsmodels.api as sm  # Alternative using statsmodels

# Load Iris dataset
iris = datasets.load_iris()
X = iris.data
y = (iris.target == 1).astype(int)  # Binary target: 1 if 'versicolor', 0 otherwise

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and fit the Probit model
kernel = DotProduct()  # Probit model typically uses a linear kernel
model = GaussianProcessClassifier(kernel=kernel, random_state=0, optimizer=None)
model.fit(X_train, y_train)

# Predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Performance Metrics
# Training performance
print("\nTraining Performance:")
print("Accuracy:", accuracy_score(y_train, y_pred_train))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))
print("Classification Report:\n", classification_report(y_train, y_pred_train))

# Testing performance
print("\nTesting Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Statsmodels implementation (for reference)
# X_train_const = sm.add_constant(X_train)
# X_test_const = sm.add_constant(X_test)
# sm_model = sm.Probit(y_train, X_train_const).fit()
# print(sm_model.summary())

##Confusion Matrix and Performance Metrics in Classification
Confusion Matrix
A confusion matrix is a table used to evaluate the performance of a classification model. For binary classification, it contains four elements:

  - True Positives (TP): Correctly predicted positive observations.
  - True Negatives (TN): Correctly predicted negative observations.
  - False Positives (FP): Incorrectly predicted positive observations (Type I error).
  - False Negatives (FN): Incorrectly predicted negative observations (Type II error).


-Performance Metrics:
Several metrics derived from the confusion matrix are used to assess a classifier's performance:

  - Accuracy: Overall, how often is the classifier correct? $$ \frac{TP + TN}{TP + TN + FP + FN} $$
  - Precision: When it predicts positive, how often is it correct? $$\frac{TP}{TP + FP} $$
  - Recall (Sensitivity): How often does it correctly predict positive out of actual positives? $$ \frac{TP}{TP + FN} $$
  - F1 Score: Harmonic mean of precision and recall. $$ 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$


1. **ROC (Receiver Operating Characteristic) Curve**:
   - **Purpose**: The ROC curve is used to visualize the performance of a classification model, particularly its ability to distinguish between classes.
   - **How it Works**: The curve plots two parameters:
     - **True Positive Rate (TPR)**, or Recall or Sensitivity, on the y-axis. TPR is the ratio of true positive predictions to the total actual positives.
     - **False Positive Rate (FPR)**, or 1-Specificity, on the x-axis. FPR is the ratio of false positive predictions to the total actual negatives.
   - **Interpretation**: A model that perfectly distinguishes between the positive and negative class will have a ROC curve that passes through the top left corner of the plot (100% sensitivity, 0% FPR). The closer the curve follows the left-hand border and then the top border of the ROC space, the more accurate the test.

2. **AUC (Area Under the ROC Curve)**:
   - **Purpose**: AUC provides a single measure of how well a model can distinguish between classes, regardless of the classification threshold.
   - **How it Works**: AUC is calculated as the area under the ROC curve.
   - **Interpretation**: An AUC of 1.0 indicates perfect classification, while an AUC of 0.5 suggests no discriminative power (equivalent to random guessing). The higher the AUC, the better the model is at predicting 0s as 0s and 1s as 1s.

3. **AUPRC (Area Under the Precision-Recall Curve)**:
   - **Purpose**: AUPRC is particularly useful for evaluating models on imbalanced datasets, where positive cases are rare compared to negative cases.
   - **How it Works**: This curve plots two parameters:
     - **Precision** (Positive Predictive Value) on the y-axis, which is the ratio of true positives to the sum of true and false positives.
     - **Recall** (True Positive Rate) on the x-axis, which is the same as TPR in the ROC curve.
   - **Interpretation**: A model with perfect precision and recall would have an AUPRC of 1. The higher the AUPRC, the better the model is at distinguishing the positive class in an imbalanced dataset.

For illustrative graphs, it's best to see them in context, as they visually represent the above explanations. Here are links to images of each:

- [ROC Curve Example](https://en.wikipedia.org/wiki/File:Roccurves.png) (Source: Wikipedia)
- [AUC Illustration](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5) (Source: Towards Data Science)
- [AUPRC Example](https://medium.com/nlp-trend-and-review-en/precision-recall-f1-score-accuracy-4d6953a2c4b2) (Source: Medium)

These graphs will give you a visual understanding of how these metrics plot model performance.

- Note:
These metrics are universally applicable across classifiers (e.g., SVM, Decision Trees, Random Forests, etc.). The choice of metric should align with the specific goals and context of the classification task, as different classifiers and problems may prioritize different aspects of performance (e.g., precision vs. recall).



In [ ]:
import numpy as np
import plotly.graph_objs as go
from sklearn.metrics import roc_curve, auc
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Step 1: Generate a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_scores = model.predict_proba(X_test)[:, 1]

# Step 2: Calculate TPR, FPR, and thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_scores)

# Step 3: Calculate AUC
roc_auc = auc(fpr, tpr)

# Step 4: Create an interactive plot
trace1 = go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC Curve (area = {roc_auc:.2f})')
trace2 = go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guess', line=dict(dash='dash'))

layout = go.Layout(title='ROC Curve and AUC',
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'),
                   showlegend=True)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objs as go
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier

# Function to generate dataset and compute ROC curve and AUC
def generate_roc_curve(completely_separable=False):
    # Step 1: Generate a synthetic dataset
    if completely_separable:
        # Completely separable case
        X = np.random.randn(1000, 2)
        y = (X[:, 0] > 0).astype(int)
    else:
        # Completely inseparable case (random guessing)
        X = np.random.randn(1000, 2)
        y = np.random.randint(0, 2, 1000)

    # Train a model
    model = RandomForestClassifier()
    model.fit(X, y)
    y_scores = model.predict_proba(X)[:, 1]

    # Step 2: Calculate TPR, FPR, and thresholds
    fpr, tpr, thresholds = roc_curve(y, y_scores)

    # Step 3: Calculate AUC
    roc_auc = auc(fpr, tpr)

    return fpr, tpr, roc_auc

# Generate ROC curves for both cases
fpr_separable, tpr_separable, auc_separable = generate_roc_curve(completely_separable=True)
fpr_inseparable, tpr_inseparable, auc_inseparable = generate_roc_curve(completely_separable=False)

# Step 4: Create an interactive plot
trace1 = go.Scatter(x=fpr_separable, y=tpr_separable, mode='lines', name=f'Completely Separable (AUC = {auc_separable:.2f})')
trace2 = go.Scatter(x=fpr_inseparable, y=tpr_inseparable, mode='lines', name=f'Completely Inseparable (AUC = {auc_inseparable:.2f})')
trace3 = go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guess', line=dict(dash='dash'))

layout = go.Layout(title='ROC Curve and AUC for Extreme Cases',
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'),
                   showlegend=True)

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()
